In [1]:
%run main.py

In [2]:
amount = 7

In [3]:
product_iri = ProductIRI("http://data.europa.eu/xsp/cn2024/100500000080")

In [4]:
user_input = UserInput(
    product_iri=product_iri,
    unit = 'kg',
    amount=amount
)

In [5]:
user_input

UserInput(product_iri=<http://data.europa.eu/xsp/cn2024/100500000080>: 1005 Maize (corn) (product), unit='kg', amount=7.0, crop_yield=None, fertilizer_amount=None, begin_date=None, end_date=None)

In [6]:
run_config = RunConfig()

In [7]:
crop = Crop(user_input = user_input, run_config = run_config)

In [8]:
resulting_demands = crop.run()

INFO: Getting master db
INFO: Getting crop yield and fertilizer amount


12:25:39 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query


INFO: Found broader match for <http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)
INFO:                                          crop_iri  \
9   http://data.europa.eu/xsp/cn2024/060011000090   
10  http://data.europa.eu/xsp/cn2024/060011000090   
11  http://data.europa.eu/xsp/cn2024/060011000090   
12  http://data.europa.eu/xsp/cn2024/060011000090   
13  http://data.europa.eu/xsp/cn2024/060011000090   
14  http://data.europa.eu/xsp/cn2024/060011000090   
15  http://data.europa.eu/xsp/cn2024/060011000090   
16  http://data.europa.eu/xsp/cn2024/060011000090   
17  http://data.europa.eu/xsp/cn2024/060011000090   

                                         fert_iri climate_type  fert_type  \
9   http://data.europa.eu/xsp/cn2024/310200000080          wet  inorganic   
10  http://data.europa.eu/xsp/cn2024/310200000080          wet  inorganic   
11  http://data.europa.eu/xsp/cn2024/310200000080          wet  inorganic   
12  http://data.europa.eu/xsp/cn20

12:25:39 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query


INFO: Found broader match for <http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)
INFO:                                         crop_iri  \
0  http://data.europa.eu/xsp/cn2024/060011000090   
1  http://data.europa.eu/xsp/cn2024/060011000090   
2  http://data.europa.eu/xsp/cn2024/060011000090   

                                        fert_iri climate_type fert_type  \
0  http://data.europa.eu/xsp/cn2024/310200000080          dry   default   
1  http://data.europa.eu/xsp/cn2024/310200000080          dry   default   
2  http://data.europa.eu/xsp/cn2024/310200000080          dry   default   

  value_type  emission_factor  emission [kg_N20/ha]  
0      value            0.005                  0.55  
1        min            0.000                  0.00  
2        max            0.011                  1.21  
INFO: Getting emission from fertilizer


wet                                          crop_iri  \
9   http://data.europa.eu/xsp/cn2024/060011000090   
10  http://data.europa.eu/xsp/cn2024/060011000090   
11  http://data.europa.eu/xsp/cn2024/060011000090   
12  http://data.europa.eu/xsp/cn2024/060011000090   
13  http://data.europa.eu/xsp/cn2024/060011000090   
14  http://data.europa.eu/xsp/cn2024/060011000090   
15  http://data.europa.eu/xsp/cn2024/060011000090   
16  http://data.europa.eu/xsp/cn2024/060011000090   
17  http://data.europa.eu/xsp/cn2024/060011000090   

                                         fert_iri climate_type  fert_type  \
9   http://data.europa.eu/xsp/cn2024/310200000080          wet  inorganic   
10  http://data.europa.eu/xsp/cn2024/310200000080          wet  inorganic   
11  http://data.europa.eu/xsp/cn2024/310200000080          wet  inorganic   
12  http://data.europa.eu/xsp/cn2024/310200000080          wet    organic   
13  http://data.europa.eu/xsp/cn2024/310200000080          wet    organic   
14 

In [ ]:
resulting_demands

# Main tst SentierModel

In [ ]:
import sentier_data_tools

In [ ]:
sentier_data_tools.__version__

In [ ]:
%run main_tst.py

# n2OToAirInorganicFertiliserDirect_dev.py

In [1]:
import n2OToAirInorganicFertiliserDirect_dev as n2o

In [2]:
import pandas as pd

In [9]:
product_iri = ProductIRI("http://data.europa.eu/xsp/cn2024/100500000080")

In [10]:
run(product_iri, 7)

Exception: File `'(product_iri,'` not found.

# function.py

In [5]:
from sentier_data_tools.iri import ProductIRI

In [6]:
import function

In [7]:
product_iri = ProductIRI("http://data.europa.eu/xsp/cn2024/100500000080")

In [4]:
product_iri.display()

'<http://data.europa.eu/xsp/cn2024/100500000080>: 1005 Maize (corn) (product)'

In [5]:
unique_product = [ product_iri.broader()[0], product_iri.broader()[1], product_iri.narrower()[0]]

09:52:08 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query
09:52:08 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query
09:52:08 [info     ] Retrieved 7 triples from https://fuseki.d-d-s.ch/skosmos/query


In [6]:
unique_product

[<http://data.europa.eu/xsp/cn2024/100021000090>: CHAPTER 10 - CEREALS (product),
 <http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product),
 <http://data.europa.eu/xsp/cn2024/100590000080>: Other (product)]

In [7]:
match_IRI = function.find_match_IRI(product_iri, unique_product)

09:52:09 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query


INFO: Found broader match for <http://data.europa.eu/xsp/cn2024/100021000090>: CHAPTER 10 - CEREALS (product)


In [16]:
df = pd.read_csv("../docs/EF.csv", sep=';')
productIRI_columns_list = ['crop_iri', 'fert_iri']

df[productIRI_columns_list] = df[productIRI_columns_list].map(lambda x : ProductIRI(x))

In [18]:
df['crop_iri'][0]

<http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)

In [19]:
df_emission_factors['crop_iri'][0]

'http://data.europa.eu/xsp/cn2024/060011000090'